# Trajectory data

In this notebook, we will explore some trajectory data, and more advanced ways of colocating it with other data sources

In [ ]:
import pathlib
from datetime import datetime
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs

## Loading data
The data we will be using is from the International Best Track Archive for Climate Stewardship(IBTrACS). This dataset contains the trajectories of past tropical cyclones, including their location, over their lifetimes.

In [ ]:
data_path = pathlib.Path("../../data_samples")

ibtracs_file = data_path / "trajectory" / "IBTrACS.NA.v04r01.nc"

assert ibtracs_file.exists, "IBTracs file not found!"

ds = xr.open_dataset(ibtracs_file)

In [ ]:
ds

Now let's isolate a single hurricane --- hurricane Sam from 2021 --- based on its storm ID:

In [ ]:
sam_sid = b"2021266N10327"
ds = ds.where(ds.sid==sam_sid, drop=True).squeeze().swap_dims(dict(date_time="time"))

The dataset also includes 360 time points to ensure all trajectories fit in the file, even though most are shorter than this. Let's also trim the trajectory to only the points with data present:

In [ ]:
ds = ds.isel(time=np.isfinite(ds.time))

In [ ]:
ds

## Plotting a trajectory

Now let's plot the trajectory on a map:

In [ ]:
fig, ax = plt.subplots(1,1,subplot_kw=dict(projection=ccrs.PlateCarree()))

ax.plot(ds.lon, ds.lat, "r-", transform=ccrs.Geodetic())
ax.plot(ds.lon[0], ds.lat[0], "ro", transform=ccrs.Geodetic())
ax.plot(ds.lon[-1], ds.lat[-1], "rx", transform=ccrs.Geodetic())
ax.coastlines()
ax.background_img()
ax.set_extent([-65, -20, 5, 60])

The trajectory is not only a series of points in space, but also in time. We need to take this into account when comparing to other data!

## Colocating data with the trajectory:
Now let's look into colocating the trajectory with another source of data. The easiest way to do this is using xarray's nearest neighbour search:

In [ ]:
# Load some ERA-5 data from the cloud:
era5 = xr.open_zarr(
    'gs://gcp-public-data-arco-era5/ar/full_37-1h-0p25deg-chunk-1.zarr-v3',
    chunks=None,
    storage_options=dict(token='anon'),
)

In [ ]:
era5

In [ ]:
sst = era5.sea_surface_temperature

In [ ]:
sst.sel(time=ds.time[0],method="nearest").plot()

In [ ]:
sst

In [ ]:
# Now let's index using the lat, lon and time from the hurricane trajectory:
sam_sst = sst.sel(time=ds.time, latitude=ds.lat%360, longitude=ds.lon%360, method="nearest")

In [ ]:
sam_sst.plot()

Nearest neighbour interpolation is simple and quick, but what if we want something a bit more precise? xarray can also be used to interpolate the values at each point along the trajectory:

In [ ]:
sam_sst_interp = sst.interp(
    coords=dict(time=ds.time, latitude=ds.lat%360, longitude=ds.lon%360), 
    method="linear"
)

In [ ]:
sam_sst_interp.plot()

This is slower as it needs to load more data, but can be more accurate depending on the use case and the resolution of the data being colocated to the trajectory (generally lower resolution data benefits more from interpolation)

## Advanced colocation
But what if we wanted to colocate some data that isn't on a nice rectilinear grid (e.g. the MODIS data from the satellite notebook), or what if we want more than just a single point? For more advanced colocation, we can use a useful tool provided by the `scikit-learn` library:

To colocate the nearest measurements, we can use a `BallTree`, which is an efficient data structure for finding nearest neighbours

In [ ]:
# First create a BallTree using the ERA5 latitude and longitude pairs
from sklearn.neighbors import BallTree
btree = BallTree(
    np.radians( # Note that we need to convert lat/lon to radians for the Haversine distance to work correctly
        np.stack(np.meshgrid(sst.latitude, sst.longitude, indexing="ij"), -1).reshape(-1,2) # BallTree needs 1d arrays for each coordinate, so reshape
    ), 
    metric = "haversine",
)

The advantage of the BallTree is that we only have to compute it once, and then finding points from it is easy and fast.

The main method to find nearest neighbours is using the `.query` method, which finds the `k` nearest neighbours (default 1). We can pass multiple points to this and find the indexes of each ones neighbours:

In [ ]:
# Now query the BallTree for the nearest neighbour to each Cloudsat lat/lon pair:
distances, indexes = btree.query(
    np.radians(np.stack([ds.lat, ds.lon], -1)), 
)

This returns an array of the 1D indexes of the nearest points, and an array of the distances (in radians) between the trajectory points and the nearest era5 points

#### Exercise:
Convert the 1D indexes back into 2D indexes using the `np.unravel_index` function. Find the EAR5 latitude and longitude of each of these points using these new indexes. Do these match the latitude and longitudes that `xarray` returned for the nearest neighbours?

In [ ]:
lat_id, lon_id = ...

Alternatively to finding a fixed number of neighbours, you can also use the BallTree to find all points within a certain distance using the `.query_radius` method.

#### Exercise:
Find the point at which Hurricane Sam reached its fastest wind speeds. Then, find all the SST values within 1000km of this point. Finally, use `xarray`'s `groupby_bins` function to see how the average SST varies with distance from the hurricane centre.

Tips:
  - use `.idxmax` to find the coordindates of a dataarray at its max value
  - convert distance to radians by dividing the distance by the radius of the Earth, convert radians to distance by multiplying by the Earth radius
  - use the keyword `return_distance=True` with `query_radius` to get distances as well as indexes, note that the arrays returned will be length one arrays containing the arrays for the queried point, so make sure to select the 0th item